In [ ]:
using Revise
using Random

import QuanticsGrids as QG
import TensorCrossInterpolation as TCI
using Test
using TCIITensorConversion
import TCIAlgorithms as TCIA

## 2D Gaussian * 2D Gaussian

In [ ]:
Random.seed!(1234)
gaussian(x, y) = exp(-0.5 * (x^2 + y^2))
R = 20
xmax = 10.0
grid = QG.DiscretizedGrid{2}(R, (-xmax, -xmax), (xmax, xmax))
grid1 = QG.DiscretizedGrid{1}(R, -xmax, xmax)
localdims = fill(4, R)
sitedims = [[2, 2] for _ in 1:R]
qf = x -> gaussian(QG.quantics_to_origcoord(grid, x)...)

pordering = TCIA.PatchOrdering(collect(1:R))

In [ ]:

expttpatches = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf, localdims), pordering; verbosity=0, maxbonddim=30
    ),
    sitedims,
)

In [ ]:
#4 patches => expttpatches has 4 elements, exptt only 1 patch
@show length(expttpatches)
@show expttpatches[1].projector
@show expttpatches[2].projector
@show expttpatches[3].projector
@show expttpatches[4].projector

In [ ]:
# all patches have maxbonddim = 26
@show TCI.linkdims(expttpatches[1].data)
@show TCI.linkdims(expttpatches[2].data)
@show TCI.linkdims(expttpatches[3].data)
@show TCI.linkdims(expttpatches[4].data)

In [ ]:
#matrix multiplication of the 2d Gaussians
product = TCIA.adaptivematmul(expttpatches, expttpatches, pordering; maxbonddim=50)

In [ ]:
#we get 8 resulting patches with a maximum bond dimension < 50
@show length(product)
for i in 1:length(product)
    @show product[i].data 
end

In [ ]:
nested_quantics(x, y) = [
            collect(p) for
            p in zip(QG.origcoord_to_quantics(grid1, x), QG.origcoord_to_quantics(grid1, y))
        ]

points = [(rand() * 10 - 5, rand() * 10 - 5) for i in 1:100];

In [ ]:
#exact solution for integration 
expproduct(x, y) =  sqrt(π) * exp(-0.5 * (x^2 + y^2)) #integrated out exp(-z^2)dz

In [ ]:
@test isapprox(
            [expproduct(p...) for p in points],
            (2xmax / 2^R) .* [product(nested_quantics(p...)) for p in points], #(2xmax/2^R) = Δx, Δy
            atol=1e-3,
        )

## $\int_{z=0}^1 (xz)*(zy)^2 dz = \frac{xy^2}{4}$

In [ ]:
Random.seed!(1234)
f1(x, y) = x*y 
f2(x, y) = (x*y)^2 
R = 20
grid = QG.DiscretizedGrid{2}(R, (0, 0), (1, 1))
grid1 = QG.DiscretizedGrid{1}(R, 0, 1)
localdims = fill(4, R)
sitedims = [[2, 2] for _ in 1:R]
qf1 = x -> f1(QG.quantics_to_origcoord(grid, x)...)
qf2 = x -> f2(QG.quantics_to_origcoord(grid, x)...)

pordering = TCIA.PatchOrdering(collect(1:R))
    

In [ ]:
tt_f1 = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf1, localdims), pordering; verbosity=0
    ),
    sitedims,
)

In [ ]:
tt_f2 = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf2, localdims), pordering; verbosity=0
    ),
    sitedims,
)

In [ ]:
# without maxbonddim constraint single tensor trains
@show length(tt_f1)
@show length(tt_f2)
@show TCI.linkdims(tt_f1[1].data)
@show TCI.linkdims(tt_f2[1].data)

In [ ]:
#now with maxbonddim = 8
# single patch
tt_f1_patches = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf1, localdims), pordering; verbosity=0, maxbonddim=8
    ),
    sitedims,
)

In [ ]:
#85 patches
tt_f2_patches = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf2, localdims), pordering; verbosity=0, maxbonddim=8
    ),
    sitedims,
)

In [ ]:
@show length(tt_f1_patches)
@show length(tt_f2_patches)
@show TCI.linkdims(tt_f1_patches[1].data)
@show TCI.linkdims(tt_f2_patches[1].data)

In [ ]:
product_without_patches = TCIA.adaptivematmul(tt_f1, tt_f2, pordering)

In [ ]:
product = TCIA.adaptivematmul(tt_f1_patches, tt_f2_patches, pordering; maxbonddim=20)

In [ ]:
# product has 85 elements
@show length(product_without_patches)
@show length(product)

In [ ]:
nested_quantics(x, y) = [
            collect(p) for
            p in zip(QG.origcoord_to_quantics(grid1, x), QG.origcoord_to_quantics(grid1, y))
        ]

points = [(rand(), rand()) for i in 1:100];
exact_product(x, y) =  x*y^2/4 #integrated (xy)*(yz)^2 dz from 0 to 1

In [ ]:
#test without patches
@test isapprox(
            [exact_product(p...) for p in points],
            (1 / 2^R) .* [product_without_patches(nested_quantics(p...)) for p in points],
            atol=1e-4,
        )

In [ ]:
#test with patches
@test isapprox(
            [exact_product(p...) for p in points],
            (1 / 2^R) .* [product(nested_quantics(p...)) for p in points],
            atol=1e-4,
        )

## product of diagonal matrices

### $x^2 * x^3$ in diagonals => problem can occur

In [ ]:
Random.seed!(1234)
f1(x, y) =  ==(x,y)*x^2 # diagonal matrix with x^2 in diagonal
f2(x, y) =  ==(x,y)*(x^3) # diagonal matrix with x^3 in diagonal
R = 5
grid = QG.InherentDiscreteGrid{2}(R, (0, 0), step=(1, 1)) # from 0 to 2^R-1 = 31
grid1 = QG.InherentDiscreteGrid{1}(R, 0, step=1)
localdims = fill(4, R)
sitedims = [[2, 2] for _ in 1:R]
qf1 = x -> f1(QG.quantics_to_origcoord(grid, x)...)
qf2 = x -> f2(QG.quantics_to_origcoord(grid, x)...)
initialpivots = [QG.origcoord_to_quantics(grid,(31,31))] #largest element
pordering = TCIA.PatchOrdering(collect(1:R))
    

In [ ]:
tt_f1 = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf1, localdims), pordering,initialpivots=initialpivots, verbosity=0
    ),
    sitedims,
)

In [ ]:
tt_f2 = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf2, localdims), pordering,initialpivots=initialpivots, verbosity=0
    ),
    sitedims,
)

In [ ]:
@show TCI.linkdims(tt_f1[1].data)
@show TCI.linkdims(tt_f2[1].data)
@show length(tt_f1)
@show length(tt_f2)

In [ ]:
#1 patch
tt_f1_patches = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf1, localdims), pordering,initialpivots=initialpivots, verbosity=0, maxbonddim=4
    ),
    sitedims,
)

In [ ]:
#10 patches
tt_f2_patches = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf2, localdims), pordering,initialpivots=initialpivots, verbosity=0, maxbonddim=4
    ),
    sitedims,
)

In [ ]:
@show length(tt_f1_patches)
@show length(tt_f2_patches)
@show TCI.linkdims(tt_f1_patches[1].data)
@show TCI.linkdims(tt_f2_patches[1].data)

In [ ]:
product_without_patches = TCIA.adaptivematmul(tt_f1, tt_f2, pordering; maxbonddim=10)

In [ ]:
product = TCIA.adaptivematmul(tt_f1_patches, tt_f2_patches, pordering; maxbonddim=10)

In [ ]:
@show length(product)
@show length(product_without_patches)

In [ ]:
nested_quantics(x, y) = [
            collect(p) for
            p in zip(QG.origcoord_to_quantics(grid1, x), QG.origcoord_to_quantics(grid1, y))
        ]

A = zeros(2^R,2^R) .+ 0.0
B=zeros(2^R,2^R) .+ 0.0
for i in 0:2^R-1
    A[i+1,i+1] =  i^2
    B[i+1,i+1] = i^3 
end
C= A*B

product_matrix = zeros(2^R,2^R) .+ 0.0
product_matrix_without_patches = zeros(2^R,2^R) .+ 0.0
for i in 0:2^R-1
    product_matrix[i+1,i+1]=product(nested_quantics(i,i))
    product_matrix_without_patches[i+1,i+1]=product_without_patches(nested_quantics(i,i))
end

In [ ]:
@show maximum(abs.(product_matrix.-C)) 
@show maximum(abs.(product_matrix_without_patches.-C)) 

In [ ]:
@test maximum(abs.(product_matrix.-C)) < 1e-5
@test maximum(abs.(product_matrix_without_patches.-C)) < 1e-5

#### check adaptiveinterpolate of diagonal matrix

here we get problems - sometimes we get 7 patches - then we get problems, if we get 10 patches it works

In [ ]:
tt_f2_matrix = zeros(2^R,2^R) .+ 0.0
for k in 1:15
    initialpivots = [QG.origcoord_to_quantics(grid,(15,15))]
    tt_f2_patches = reshape(
        TCIA.adaptiveinterpolate(
            TCIA.makeprojectable(Float64, qf2, localdims), pordering,initialpivots=initialpivots, verbosity=0, maxbonddim=4
        ),
        sitedims,
    )

    f2_matrix = zeros(2^R,2^R) .+ 0.0
    for i in 0:2^R-1
        f2_matrix[i+1,i+1] =  i^3
    end

    tt_f2_matrix = zeros(2^R,2^R) .+ 0.0
    for i in 0:2^R-1
        tt_f2_matrix[i+1,i+1]=tt_f2_patches(nested_quantics(i,i))
    end

    @show maximum(abs.(f2_matrix.-tt_f2_matrix))
    @show length(tt_f2_patches)
end

In [ ]:
for i in 1:32
    println(tt_f2_matrix[i,i])
end

## diagonal matrices

In [ ]:
Random.seed!(1234)
f1(x, y) =  ==(x,y)*x^2 # diagonal matrix with x^2 in diagonal
f2(x, y) =  ==(x,y)*(x^3) # diagonal matrix with x^3 in diagonal
R = 5
grid = QG.InherentDiscreteGrid{2}(R, (0, 0), step=(1, 1)) # from 0 to 2^R-1 = 31
grid1 = QG.InherentDiscreteGrid{1}(R, 0, step=1)
localdims = fill(4, R)
sitedims = [[2, 2] for _ in 1:R]
qf1 = x -> f1(QG.quantics_to_origcoord(grid, x)...)
qf2 = x -> f2(QG.quantics_to_origcoord(grid, x)...)
initialpivots = [QG.origcoord_to_quantics(grid,(2^R-1,2^R-1))] #largest element
pordering = TCIA.PatchOrdering(collect(1:R))
  
tt_f1 = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf1, localdims), pordering,initialpivots=initialpivots, verbosity=0
    ),
    sitedims,
)

tt_f2 = reshape(
    TCIA.adaptiveinterpolate(
        TCIA.makeprojectable(Float64, qf2, localdims), pordering,initialpivots=initialpivots, verbosity=0
    ),
    sitedims,
)

In [ ]:
tt_f1_projected = TCIA.ProjTTContainer([
    TCIA.project(tt_f1[1], p) for p in [
        TCIA.Projector([[1, 1], [0, 0], [0, 0], [0, 0], [0, 0]], sitedims),
        TCIA.Projector([[2, 2], [0, 0], [0, 0], [0, 0], [0, 0]], sitedims),
    ]
])

In [ ]:
tt_f2_projected = TCIA.ProjTTContainer([
    TCIA.project(tt_f2[1], p) for p in [
        TCIA.Projector([[1, 1], [0, 0], [0, 0], [0, 0], [0, 0]], sitedims),
        TCIA.Projector([[2, 2], [0, 0], [0, 0], [0, 0], [0, 0]], sitedims),
    ]
])

In [ ]:
product = TCIA.adaptivematmul(tt_f1_projected, tt_f2_projected, pordering; maxbonddim=5)
product_without_patches = TCIA.adaptivematmul(tt_f1_projected, tt_f2_projected, pordering; maxbonddim=10)

In [ ]:
# two patches
@show length(product)
@show length(product_without_patches)
for i in 1:length(product)
    @show TCI.linkdims(product[i].data)
end
@show TCI.linkdims(product_without_patches[1].data)

In [ ]:
nested_quantics(x, y) = [
            collect(p) for
            p in zip(QG.origcoord_to_quantics(grid1, x), QG.origcoord_to_quantics(grid1, y))
        ]

A = zeros(2^R,2^R) .+ 0.0
B=zeros(2^R,2^R) .+ 0.0
for i in 0:2^R-1
    A[i+1,i+1] =  i^2
    B[i+1,i+1] = i^3 
end
C= A*B

product_matrix = zeros(2^R,2^R) .+ 0.0
product_matrix_without_patches = zeros(2^R,2^R) .+ 0.0
for i in 0:2^R-1, j in 0:2^R-1
    product_matrix[i+1,j+1]=product(nested_quantics(i,j))
    product_matrix_without_patches[i+1,j+1]=product_without_patches(nested_quantics(i,j))
end

In [ ]:
@show maximum(abs.(product_matrix.-C)) 
@show maximum(abs.(product_matrix_without_patches.-C)) 
@test maximum(abs.(product_matrix.-C)) < 1e-5
@test maximum(abs.(product_matrix_without_patches.-C)) < 1e-5
